In [1]:
import numpy as np
import pprint
from operator import itemgetter
from mdp_matrix import GridWorld, WindyGridCliffMazeWorld, StochasticGridWorld
from sarsa import sarsa
from expected_sarsa import expected_sarsa
from double_sarsa import double_sarsa
from double_expected_sarsa import double_expected_sarsa

In [2]:
start_state = [0, 0]

test_rewards = [[i, j, -1] for i in range(5) for j in range(5)]
test_rewards[2] = [0, 2, 1]
test_rewards[23] = [4,3, 1]

gw = GridWorld(5, test_rewards, terminal_states=[2, 23] )

In [ ]:
n_step_sarsa(gw, 10)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead


In [6]:
import numpy as np
import sys
    
def behaviour_policy(Q, s, A, epsilon=.1):
    """
    Recall that off-policy learning is learning the value function for
    one policy, \pi, while following another policy, \mu. Often, \pi is
    the greedy policy for the current action-value-function estimate, 
    and \mu is a more exploratory policy, perhaps \epsilon-greedy.
    In order to use the data from \pi we must take into account the 
    difference between the two policies, using their relative
    probability of taking the actions that were taken.
    """
    if np.random.random_sample() <= epsilon:
        a = np.random.random_integers(0, A-1)
    else:
        a = np.argmax(Q[s][:])
    return a

def target_policy(Q, s):
    return np.argmax(Q[s][:])

def n_step_sarsa(mdp, max_episode, alpha = 0.1, gamma = 0.9, epsilon = 0.1, n = 10):
    # Initialization
    Q = [[0 for i in range(mdp.A)] for j in range(mdp.S)]
    old_Q = Q
    n_episode = 0
    rewards_per_episode = []
    Q_variances = []
    max_reward = 0
    total_reward = 0

    while n_episode < max_episode:
        # If there's no starting state, just start at state 0
        try:
            s = mdp.initial_state # Initialize s, starting state
        except AttributeError:
            s = 0

        # initializations
        T = sys.maxint
        tau = 0
        t = -1
        stored_actions = {}
        stored_rewards = {}
        stored_states = {}

        # With prob epsilon, pick a random action
            
        stored_actions[0] = behaviour_policy(Q, s, mdp.A)
        reward_for_episode = 0
        
        while tau < (T-1):
            t += 1
            if t < T:
                # take action A_t
                
                # Observe and store the next reward R_{t+1} and next state S_{t+1}
                st1 = np.random.choice(range(mdp.S), p = mdp.T[s, stored_actions[t], :])
                rt1 = mdp.R[st1]
                
                stored_rewards[t] = rt1
                stored_states[t] = st1
                
                # TODO: is this the right place to put this?
                total_reward += rt1
                reward_for_episode += rt1
                
                # if s_{t+1} terminal
                if mdp.is_terminal(st1):
                    T = t + 1
                else:
                    stored_actions[t+1] = behaviour_policy(Q, s, mdp.A)
            
            tau = t - n
            if tau >= 0:
                # product from i = tau+1 to min(tau+n-1, T-1) \pi(A_i | S_i) / \mu(A_i|S_i)
                rho = np.prod([(stored_actions[k] == behaviour_policy(Q, stored_states[k], mdp.A)) / ((stored_actions[k] != np.argmax(Q[stored_states[k]][:]))*epsilon*1/mdp.A + (stored_actions[k] == np.argmax(Q[stored_states[k]][:]))*(epsilon*1/mdp.A + (1-epsilon)))  for k in range(tau+1, min(tau+n-1, T-1)+1)])
                
                G = np.sum([gamma**(i-tau-1) * stored_rewards[i] for i in range(tau+1, min(tau+n, T)+1)])
                
                if tau + n < T:
                    G = G + gamma**n * Q[stored_states[tau+n]][stored_actions[tau+n]]
                s_tau = stored_states[tau]
                a_tau = stored_actions[tau]
                Q[s_tau][a_tau] += alpha * rho * (G - Q[s_tau][a_tau])
                
                print tau
                print T
                # if pi is being learned, ensure that pi(.|S_tau) is \epsilon-greedy wrt Q
          


        if reward_for_episode > max_reward:
            max_reward = reward_for_episode

        rewards_per_episode.append(reward_for_episode)
        Q_variances.append(np.var(Q))
        
        #TODO: should we instead do an on-policy run here to calculate the
        # average reward for the episode?

        n_episode += 1
        print n_episode
    return Q, total_reward/max_episode, max_reward, rewards_per_episode, Q_variances
